# K-Nearest step by step

## Load dataset

In [4]:
# I decided to go with the breast cancer dataset
# https://www.kaggle.com/uciml/breast-cancer-wisconsin-data
from sklearn.datasets import load_breast_cancer

## Features and classes

##### Features are computed from a digitized image of a fine needle aspirate of a breast mass. They describe characteristics of the cell nuclei present in the image.
a) radius (mean of distances from center to points on the perimeter)

b) texture (standard deviation of gray-scale values) 

c) perimeter 

d) area 

e) smoothness (local variation in radius lengths) 

f) compactness (perimeter^2 / area - 1.0) 

g) concavity (severity of concave portions of the contour)

h) concave points (number of concave portions of the contour) 

i) symmetry 

j) fractal dimension ("coastline approximation" - 1)

**The mean, standard error and "worst" or largest (mean of the three largest values) of these features were computed for each image, resulting in 30 features:**


- 1 - mean radius
- 2 - mean texture
- 3 - mean perimeter
- 4 - mean area
- 5 - mean smoothness
- 6 - mean compactness
- 7 - mean concavity
- 8 - mean concave points
- 9 - mean symmetry
- 10 - mean fractal dimension
- 11 - radius error
- 12 - texture error
- 13 - perimeter error
- 14 - area error
- 15 - smoothness error
- 16 - compactness error
- 17 - concavity error
- 18 - concave points error
- 19 - symmetry error
- 20 - fractal dimension error
- 21 - worst radius
- 22 - worst texture
- 23 - worst perimeter
- 24 - worst area
- 25 - worst smoothness
- 26 - worst compactness
- 27 - worst concavity
- 28 - worst concave points
- 29 - worst symmetry
- 30 - worst fractal dimension 

##### the diagnosis of breast tissues
**2 classes:**
- malignant
- benign

In [6]:
import numpy as np

In [7]:
df = load_breast_cancer()

In [14]:
df_data = 

{'data': array([[1.799e+01, 1.038e+01, 1.228e+02, ..., 2.654e-01, 4.601e-01,
         1.189e-01],
        [2.057e+01, 1.777e+01, 1.329e+02, ..., 1.860e-01, 2.750e-01,
         8.902e-02],
        [1.969e+01, 2.125e+01, 1.300e+02, ..., 2.430e-01, 3.613e-01,
         8.758e-02],
        ...,
        [1.660e+01, 2.808e+01, 1.083e+02, ..., 1.418e-01, 2.218e-01,
         7.820e-02],
        [2.060e+01, 2.933e+01, 1.401e+02, ..., 2.650e-01, 4.087e-01,
         1.240e-01],
        [7.760e+00, 2.454e+01, 4.792e+01, ..., 0.000e+00, 2.871e-01,
         7.039e-02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0

In [36]:
%time
result = []

class Features():
    def __init__(self, mean_radius, mean_texture, mean_perimeter, mean_area):
        self.mean_radius = mean_radius
        self.mean_texture = mean_texture
        self.mean_perimeter = mean_perimeter
        self.mean_area = mean_area

class Target():
    def __init__(self, malignant, benign):
        self.malignant = malignant
        self.benign = benign

def get_data(result=[]):
    for ii, row in enumerate(df['data']):
        result.append(Features(float(row[0]), float(row[1]), float(row[2]), float(row[3])))

    return result



get_data(result)


CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.01 µs


In [40]:
for ii in result:
    print(f'mean radius:{ii.mean_radius}, mean texture:{ii.mean_texture}, {ii.mean_perimeter}, {ii.mean_area}')

mean radius:17.99, mean texture:10.38, 122.8, 1001.0
mean radius:20.57, mean texture:17.77, 132.9, 1326.0
mean radius:19.69, mean texture:21.25, 130.0, 1203.0
mean radius:11.42, mean texture:20.38, 77.58, 386.1
mean radius:20.29, mean texture:14.34, 135.1, 1297.0
mean radius:12.45, mean texture:15.7, 82.57, 477.1
mean radius:18.25, mean texture:19.98, 119.6, 1040.0
mean radius:13.71, mean texture:20.83, 90.2, 577.9
mean radius:13.0, mean texture:21.82, 87.5, 519.8
mean radius:12.46, mean texture:24.04, 83.97, 475.9
mean radius:16.02, mean texture:23.24, 102.7, 797.8
mean radius:15.78, mean texture:17.89, 103.6, 781.0
mean radius:19.17, mean texture:24.8, 132.4, 1123.0
mean radius:15.85, mean texture:23.95, 103.7, 782.7
mean radius:13.73, mean texture:22.61, 93.6, 578.3
mean radius:14.54, mean texture:27.54, 96.73, 658.8
mean radius:14.68, mean texture:20.13, 94.74, 684.5
mean radius:16.13, mean texture:20.68, 108.1, 798.8
mean radius:19.81, mean texture:22.15, 130.0, 1260.0
mean radius

In [52]:
f = [ii.mean_radius for ii in result]
print(f[0:10])

[17.99, 20.57, 19.69, 11.42, 20.29, 12.45, 18.25, 13.71, 13.0, 12.46]
